# Example Pyllelic Use-Case Notebook

## Background

This notebook illustrates the import and use of `pyllelic` in a jupyter environment.

See https://github.com/Paradoxdruid/pyllelic for further details.

## Pre-setup

### Obtaining fastq data

We can download rrbs (reduced representation bisulfite sequencing) data from the Encode project:
http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeHaibMethylRrbs/

Those files are in unaligned fastq format.  We will need to align these to a reference human genome.

### Aligning reads (using process.py)

To align reads, we'll use bowtie2 and samtools (through its pysam wrapper).

First, we need to download a genomic index sequence: http://hgdownload.soe.ucsc.edu/goldenPath/hg19

In [ ]:
# Processing imports
# from pathlib import Path

In [ ]:
# Set up file paths
# index = Path(
#     "/home/andrew/allellic/hg19.p13.plusMT.no_alt_analysis_set//hg19.p13.plusMT.no_alt_analysis_set"
# )
# fastq = Path("/home/andrew/allellic/wgEncodeHaibMethylRrbsU87HaibRawDataRep1.fastq.gz")

**WARNING:** The next command is processor, RAM, and time intensive, and only needs to be run once!

In [ ]:
# Convert fastq to bam
# pyllelic.process.bowtie2_fastq_to_bam(index={bowtie_index_filename_without_suffix},
#                                       fastq={fastq_file_name},
#                                       cores=6)

Notes:

* cores is number of processor cores, adjust for your system
* instead of `out.bam` use a filename that encodes cell-line and tissue.  Our convention is: `fh_CELLLINE_TISSUE.TERT.bam`

Next, we need to sort and index the bam file using samtools functions.

In [ ]:
# Sort the bamfile
# bamfile = Path("/home/andrew/allellic/wgEncodeHaibMethylRrbsU87HaibRawDataRep1.bam")
# pyllelic.process_pysam_sort(bamfile)

In [ ]:
# Create an index of the sorted bamfile
# sorted_bam = Path("")
# pyllelic.process_pysam_index(b)

Now, that sorted file (again, rename to capture cell-line and tissue info) is ready to be put in the `test` folder for analysis by pyllelic!

## Set-up

In [1]:
import pyllelic

In [6]:
# set up your disk location:
# base_path should be the directory we'll do our work in
# make a sub-directory under base_path with a folder named "test"
# and put the .bam and .bai files in "test"

# OSX setup
# pyllelic.set_up_env_variables(
#     base_path="/Users/abonham/documents/test_allelic/",
#     prom_file="TERT-promoter-genomic-sequence.txt",
#     prom_start="1293000",
#     prom_end="1296000",
#     chrom="5",
#     offset=1298163,
# )

# Windows set-up
pyllelic.set_up_env_variables(
    base_path="/home/andrew/allellic/",
    prom_file="TERT-promoter-genomic-sequence.txt",
    prom_start="1293000",
    prom_end="1296000",
    chrom="chr5",
    offset=1298163,
)

## Main Parsing Functions

In [7]:
files_set = pyllelic.make_list_of_bam_files()  # finds bam files

In [8]:
# Uncomment for debugging:
files_set

['fh_SORTED_TISSUE.TERT.bam']

In [20]:
# index bam and creates bam_output folders/files
# set process False to skip writing output files if they already exist
positions = pyllelic.index_and_fetch(files_set, process=True)

  0%|          | 0/1 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute 'seqB'

In [10]:
# Turn off pretty printing to see position list better
%pprint

Pretty printing has been turned OFF


In [11]:
# Uncomment for debugging:
positions

['1295680', '1295903']

In [12]:
# Turn back on pretty printing
%pprint

Pretty printing has been turned ON


In [13]:
# Only needs to be run once, generates static files
# pyllelic.genome_parsing()

# Can also take sub-list of directories to process
# pyllelic.genome_parsing([pyllelic.config.bam_directory / "fh_BONHAM_TISSUE.TERT.bam"])

In [14]:
cell_types = pyllelic.extract_cell_types(files_set)

In [15]:
# Uncomment for debugging
cell_types

['SORTED']

In [16]:
# Set filename to whatever you want
df_list = pyllelic.run_quma_and_compile_list_of_df(
    cell_types, "test2.xlsx",
    run_quma=True,
)  # to skip quma: , run_quma=False)

Cell Lines:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
# Uncomment for debugging
df_list.keys()

dict_keys(['SORTED'])

In [19]:
df_list["SORTED"]

,1295903,1295680
0,111,1
1,1111,11
2,1111,11
3,1111,11
4,1111,11
5,1111,11
6,1111,11
7,1111,11
8,1111,11
9,1111,11


In [ ]:
means = pyllelic.process_means(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
means

In [ ]:
modes = pyllelic.process_modes(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
modes

In [ ]:
diff = pyllelic.find_diffs(means, modes)

In [ ]:
# Uncomment for debugging
diff

## Write Output to excel files

In [ ]:
# Set the filename to whatever you want
pyllelic.write_means_modes_diffs(means, modes, diff, "Test1")

## Visualizing Data

In [ ]:
final_data = pyllelic.pd.read_excel(
    pyllelic.config.base_directory.joinpath("Test1_diff.xlsx"),
    dtype=str,
    index_col=0,
    engine="openpyxl",
)

In [ ]:
final_data

In [ ]:
individual_data = pyllelic.return_individual_data(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
individual_data

In [ ]:
individual_data.loc["NCIH196", "1295937"]

In [ ]:
individual_data.loc["SORTED"]["1295680"]

In [ ]:
individual_data.loc["CALU1"]

In [ ]:
pyllelic.histogram(individual_data, "SORTED", "1295680")

In [ ]:
pyllelic.histogram(individual_data, "SORTED", "1295903")

In [ ]:
pyllelic.histogram(individual_data, "SW1710", "1295089")

In [ ]:
pyllelic.histogram(individual_data, "CALU1", "1295937")

In [ ]:
pyllelic.histogram(individual_data, "NCIH196", "1295937")

In [ ]:
pyllelic.histogram(individual_data, "NCIH196", "1294945")

In [ ]:
final_data.loc["SW1710"]

## Statistical Tests for Normality

In [ ]:
pyllelic.summarize_allelic_data(individual_data, diff)